In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re

In [ ]:
answers = pd.read_csv('./MTC508/discussion_answers.csv')

In [ ]:
questions = pd.read_csv('./MTC508/discussion_questions.csv')

In [56]:
answers.head()

,discussion_answer_id,umich_discussions_user_id,course_id,discussion_answer_content,discussion_question_id,discussion_answer_parent_discussion_answer_id,discussion_answer_created_ts,discussion_answer_updated_ts,on_demand_session_id,course_branch_id
0,ZroNsxc4Ee2C5xKrTYcc7w,1a77c68d3e75f92e3eb7addd5b0e99875a3cee27,H2xU8YQeEeqO7w5jjw7X2w,<text>The event I picked was the treaty betwee...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-08 16:37:32.744,2022-08-08 16:37:32.744,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
1,2pQvXRv3Ee2FOwrnFlqg5Q,db6d799619ffeccbf051e29edaa465bc3bef5b18,H2xU8YQeEeqO7w5jjw7X2w,"<text>The government created this ""End of Trea...",Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-14 17:38:05.638,2022-08-14 17:38:05.638,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
2,2SAgMAbiEe2KXA5zGGl7GQ,5a337f5e59a38095ed18d4cc487f3d1f75a78ef6,H2xU8YQeEeqO7w5jjw7X2w,<text>I actually took a look at the 1968 congr...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-07-18 21:44:49.35,2022-07-25 02:11:00.708,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
3,U_HKABXXEe2toRLQeik5bQ,9233f9805b20b5b77a61987c226185a772dbf397,H2xU8YQeEeqO7w5jjw7X2w,<text>The postal act of 1792 established the r...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-06 22:30:08.888,2022-08-06 22:30:08.888,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
4,r1N-_RXXEe2KXA5zGGl7GQ,9233f9805b20b5b77a61987c226185a772dbf397,H2xU8YQeEeqO7w5jjw7X2w,<text>I totally agree. The IRA should absolutl...,Loj5KgVZEe21VQ4xyvwaNw,l3V4WwxIEe2FkAoJNY_lJQ,2022-08-06 22:32:42.20,2022-08-06 22:32:42.20,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ


In [ ]:
questions.head()

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
def remove_html_tags(text):
    clean_text = re.sub(r'<[^>]*>', '', text)  # Remove HTML tags
    clean_text = re.sub(r'[^\w\s]', '', clean_text)  # Remove special characters
    return clean_text

In [ ]:
sample = remove_html_tags(questions['discussion_question_details'][1])

In [ ]:
embeddings = model.encode([sample])
print(embeddings)

In [ ]:
len(embeddings[0])

In [ ]:
users = pd.read_csv('MTC508/users.csv')

In [ ]:
answers.shape

In [ ]:
users.shape

In [80]:
questions['discussion_question_details'] = questions['discussion_question_details'].astype(str)
questions['cleaned'] = questions['discussion_question_details'].apply(remove_html_tags)
questions['cleaned']
question_embeddings = model.encode(list(questions['cleaned']))

In [85]:
embeddings_df = pd.DataFrame(question_embeddings)
new_column_names = {old_column_name: f"q{old_column_name}" for old_column_name in range(768)}
embeddings_df.rename(columns=new_column_names, inplace=True)
embeddings_df['student_ids'] = questions['umich_discussions_user_id']

In [90]:
embeddings_df['student_ids'].value_counts()

student_ids
cc0c2da21abe4aa86470019c4e7c2640c25768ce    86
ca76161d33e80abef199b953d93265eb905d3363     2
61876a5bdcd4d02e3b498c13f4588f7612011f5b     2
a5b7c8cb596ca7d3315cdb01a06f4e283e1cced9     2
ad7b52853626b66c0d18ba8a16d6d68a557799f7     2
9134b58fe4f9c88cc1dabf69f1737a4f6f8b0491     2
661b6cb3c4003574de5608201bb473236640e68b     2
8baa9b491e71abebba1aae17a22153db4406ee71     2
5a337f5e59a38095ed18d4cc487f3d1f75a78ef6     2
2534d1e40298e013cf08f96dcd072b3e1c386687     2
16628f1d9b42b59646a0a1f0b564463af315e4a8     1
8c59c023a0de84038905a2868534c31cb4f88df4     1
12c2cfa452d9010c41340af3de0b82a6e689ec50     1
c20ceabd903ab64aa6cf03444ab227e8a9af40ff     1
001a56372a33d7cfc1590ba1e87420d63b61f64d     1
2246b3c08e2f3ee5d17f125ccc70baf3691667f9     1
7e04ef837ee9dadf81f9f2ad645db20da8a6a946     1
d5f70daff6af38c95005dcd05e597709c76df620     1
a65eeaa62965cd9245ba3a2acb80014982c021cd     1
396d32322ab12606e68b57766feec33e7bb408a7     1
a6f1bc29cf67c9b620d7463977cf0718b06c957e     1
b

In [119]:
grouped_questions = embeddings_df.groupby('student_ids').mean().reset_index()

In [120]:
grouped_questions.head()

,student_ids,q0,q1,q2,q3,q4,q5,q6,q7,q8,...,q758,q759,q760,q761,q762,q763,q764,q765,q766,q767
0,001a56372a33d7cfc1590ba1e87420d63b61f64d,0.021349,0.046682,0.017386,-0.021480,-0.072024,-0.030376,0.011503,-0.005623,0.022912,...,-0.013997,-0.057977,-0.073041,-0.024940,0.004183,0.050773,0.001498,-0.014608,0.048054,0.055343
1,04d598c7163af36ce85b21f3a136ef51ec7e4987,-0.049821,0.010487,-0.034307,-0.050995,0.026423,0.025967,0.077050,-0.011666,-0.002327,...,0.031109,0.001926,0.018485,0.013437,0.011395,-0.047218,-0.002231,0.076511,-0.039926,-0.011729
2,0884c670a5564e10524689beb523134f6f3db852,-0.040793,-0.027210,-0.004982,-0.022387,-0.013239,-0.010763,-0.022427,-0.006788,-0.012499,...,-0.043791,0.012975,-0.020288,-0.000733,-0.050783,-0.037656,0.016370,0.002353,-0.016064,-0.078920
3,12a0fc8a79beba2e40564d8be574a21706063ac7,0.013746,-0.053411,0.012271,0.008901,-0.041837,-0.010008,0.017253,0.014006,-0.078884,...,-0.030849,-0.008875,-0.049674,-0.002761,0.015663,-0.032675,0.002338,-0.017309,-0.032986,-0.018049
4,12c2cfa452d9010c41340af3de0b82a6e689ec50,-0.025188,-0.055655,-0.027111,-0.035621,-0.008580,0.062135,0.010136,-0.005559,0.037068,...,-0.034064,-0.006982,0.014783,-0.029736,-0.003789,0.011277,-0.013991,0.005936,0.002950,-0.018658


In [137]:
preprocessed = pd.read_csv('data/preprocessed.csv')

In [138]:
preprocessed = pd.merge(preprocessed, grouped_questions, left_on='umich_user_id', right_on='student_ids', how='left')
preprocessed.fillna(0, inplace=True)


In [139]:
preprocessed

,Unnamed: 0,umich_user_id,answer_count,average_answer_length,total_votes_given_answers,total_votes_received_answers,question_count,average_question_length,total_votes_given_questions,total_votes_received_questions,...,q758,q759,q760,q761,q762,q763,q764,q765,q766,q767
0,0,d8494a39cae7ac5417d491df43f8296a285b3cd6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,9c3e969ace3d1278a5e3ad42fa80dfc20379aaf5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,c91dab5c98bead9e683d15e9e2cc2c4584a8e626,10.0,797.200000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,2246b3c08e2f3ee5d17f125ccc70baf3691667f9,5.0,589.000000,1.0,1.0,1.0,573.0,0.0,4.0,...,-0.022409,-0.015020,0.009531,0.007608,-0.022088,0.001710,-0.007233,0.024724,0.032601,-0.020934
4,4,1f8331ad9f0125b0fc85a1c4ff28f9c198b6fca2,11.0,300.818182,3.0,3.0,0.0,0.0,0.0,3.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,1127,5b49d0234b44a13c4621338500f07c00769012dc,14.0,532.071429,0.0,1.0,1.0,195.0,0.0,0.0,...,-0.019878,-0.008316,-0.045150,0.053084,0.004477,0.012517,0.012616,-0.009486,-0.033686,-0.017639
1128,1128,594516421fffd9370c844dd58755ef6306fb898a,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1129,1129,e66e909487d3b7cad09fd0fdee1f0c233f00d26b,9.0,894.111111,0.0,4.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1130,1130,2137c8a34861aacfb52c049f444651312ef18b30,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [140]:
answers['discussion_answer_content'] = answers['discussion_answer_content'].astype(str)
answers['cleaned'] = answers['discussion_answer_content'].apply(remove_html_tags)
answer_embeddings = model.encode(list(answers['cleaned']))

AttributeError: 'LogisticRegression' object has no attribute 'encode'

In [141]:
answer_embeddings_df = pd.DataFrame(answer_embeddings)
new_column_names = {old_column_name: f"a{old_column_name}" for old_column_name in range(768)}
answer_embeddings_df.rename(columns=new_column_names, inplace=True)
answer_embeddings_df['student_ids'] = answers['umich_discussions_user_id']

answer_embeddings_df['student_ids'] = answer_embeddings_df['student_ids'].astype(str)
grouped_answers = answer_embeddings_df.groupby('student_ids').mean()

In [142]:
grouped_answers

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a758,a759,a760,a761,a762,a763,a764,a765,a766,a767
student_ids,,,,,,,,,,,,,,,,,,,,,
001a56372a33d7cfc1590ba1e87420d63b61f64d,-0.013522,0.065869,-0.008406,-0.012171,-0.001225,0.009266,0.010336,-0.020971,-0.002710,0.009488,...,0.008878,0.011994,-0.044760,0.004807,-0.003599,-0.022756,-0.004345,0.012866,0.031391,0.001870
001bce8241adfa1367c49360b5e5afd26feaeb6b,-0.011922,0.056499,-0.009097,0.007348,-0.009611,-0.001009,0.016043,-0.020087,-0.005588,-0.001766,...,-0.003825,0.009689,-0.039302,-0.004520,0.005158,-0.030390,-0.010249,0.010043,0.017340,0.018207
00b19bdf6490aafdacd3ed12d7d0257aa85a625f,-0.030451,0.074735,-0.011374,0.009536,-0.011005,0.008375,-0.001221,-0.026494,-0.029846,-0.012688,...,0.006727,0.017325,-0.050822,-0.005253,-0.002781,-0.004111,-0.013904,0.014792,0.028416,0.015414
00d5603b292c99b421a456835c9e5c974cc03cab,0.013595,0.031607,-0.011988,-0.004184,-0.040317,-0.026964,0.004294,-0.000314,-0.010740,0.007682,...,0.027680,-0.004390,-0.073710,0.013046,0.011822,-0.031390,0.010734,0.022881,-0.011338,0.029776
00dd021cd363b0c820ae9b3e3db17f02968d3f17,-0.009629,0.045902,0.009146,0.005806,0.012217,0.004037,0.007441,0.002885,0.025685,-0.014931,...,0.023779,0.018937,-0.052454,-0.007236,-0.002255,-0.074608,0.024336,0.022291,0.032729,0.038856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fde4c80cbda713d84f33670a3183117966d5df78,-0.021987,0.059480,-0.001000,0.002860,-0.015259,-0.002128,0.007083,-0.014792,-0.011827,-0.008434,...,-0.002459,-0.002580,-0.067217,-0.007332,-0.003163,-0.013836,-0.016976,0.009667,0.023607,0.022680
fe59afeafcfda6d4e276522f331207020bd624ff,-0.030366,0.078952,0.004165,-0.003330,-0.016270,0.007893,0.002414,-0.015942,0.001615,0.001683,...,0.016812,0.011550,-0.078482,-0.003282,-0.018209,-0.015058,-0.011848,0.015414,0.022661,0.025613
fe612e0709c05d95fb0dfa0714b485f8e32dfd5a,-0.007234,0.061073,0.001753,0.011274,-0.013498,0.012612,-0.015589,-0.007301,-0.001876,-0.010041,...,0.038757,0.012477,-0.052164,0.000332,-0.006088,-0.030556,0.000184,0.004700,0.020257,0.017000


In [143]:
preprocessed = pd.merge(preprocessed, grouped_answers, left_on='umich_user_id', right_on='student_ids', how='left')

In [144]:
preprocessed_filled = preprocessed.fillna(0)

In [145]:
na_dropped = preprocessed.dropna()

In [146]:
na_dropped

,Unnamed: 0,umich_user_id,answer_count,average_answer_length,total_votes_given_answers,total_votes_received_answers,question_count,average_question_length,total_votes_given_questions,total_votes_received_questions,...,a758,a759,a760,a761,a762,a763,a764,a765,a766,a767
2,2,c91dab5c98bead9e683d15e9e2cc2c4584a8e626,10.0,797.200000,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.010755,0.018048,-0.067982,-0.010930,-0.008356,-0.007772,-0.006473,0.008379,0.006640,-0.005703
3,3,2246b3c08e2f3ee5d17f125ccc70baf3691667f9,5.0,589.000000,1.0,1.0,1.0,573.0,0.0,4.0,...,0.001105,0.008145,-0.050516,0.003159,-0.005324,-0.019285,-0.009427,0.008880,0.013980,0.018785
4,4,1f8331ad9f0125b0fc85a1c4ff28f9c198b6fca2,11.0,300.818182,3.0,3.0,0.0,0.0,0.0,3.0,...,0.000683,0.024056,-0.053009,0.000660,0.005344,-0.008130,-0.020281,0.003803,0.007443,0.014821
6,6,83d53891c9fb20345931f165fd8cea57f9463a18,10.0,489.100000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011582,0.007623,-0.051470,-0.003476,-0.000205,-0.023117,-0.012618,0.012937,0.027619,0.016025
7,7,bac3cb1f386199a136172ad27d399925669667f2,7.0,1229.714286,0.0,3.0,0.0,0.0,0.0,0.0,...,0.012215,0.016447,-0.066674,0.004363,-0.013111,-0.019899,-0.014620,0.015653,0.028271,0.026160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,1124,ad51dd7476f641ea8562958b784af1d02963f7ed,3.0,328.333333,0.0,1.0,0.0,0.0,0.0,0.0,...,0.017158,-0.007903,-0.058705,0.035403,-0.012527,-0.037345,0.008951,0.004818,0.047732,0.016271
1125,1125,33e70b9df851650f36d4c399153afff7fb987caa,1.0,62.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050693,0.004874,0.038248,0.045999,-0.005142,0.033084,-0.026581,0.049224,-0.037998,0.022180
1127,1127,5b49d0234b44a13c4621338500f07c00769012dc,14.0,532.071429,0.0,1.0,1.0,195.0,0.0,0.0,...,0.008257,0.003782,-0.043130,-0.003785,0.002380,-0.010967,0.000069,0.001155,0.031674,0.012125
1129,1129,e66e909487d3b7cad09fd0fdee1f0c233f00d26b,9.0,894.111111,0.0,4.0,0.0,0.0,0.0,0.0,...,-0.002416,0.018168,-0.063610,0.004872,-0.012486,-0.008605,-0.016720,0.005575,0.022830,0.025355


In [147]:
lst = []

for i in range(768):
    lst.append('q' + str(i))
    lst.append('a' + str(i))

X = na_dropped[lst]
y = na_dropped['completed']


In [148]:
X.head()

,q0,a0,q1,a1,q2,a2,q3,a3,q4,a4,...,q763,a763,q764,a764,q765,a765,q766,a766,q767,a767
2,0.000000,-0.011006,0.000000,0.058872,0.000000,-0.005551,0.000000,0.003791,0.000000,-0.008598,...,0.00000,-0.007772,0.000000,-0.006473,0.000000,0.008379,0.000000,0.006640,0.000000,-0.005703
3,-0.035905,0.005975,0.008703,0.042322,-0.017534,0.009805,-0.024428,-0.013229,0.029848,0.002578,...,0.00171,-0.019285,-0.007233,-0.009427,0.024724,0.008880,0.032601,0.013980,-0.020934,0.018785
4,0.000000,-0.010419,0.000000,0.064818,0.000000,0.007832,0.000000,-0.005894,0.000000,-0.014784,...,0.00000,-0.008130,0.000000,-0.020281,0.000000,0.003803,0.000000,0.007443,0.000000,0.014821
6,0.000000,-0.001802,0.000000,0.051032,0.000000,0.002412,0.000000,0.000003,0.000000,-0.026137,...,0.00000,-0.023117,0.000000,-0.012618,0.000000,0.012937,0.000000,0.027619,0.000000,0.016025
7,0.000000,-0.046393,0.000000,0.047491,0.000000,-0.013372,0.000000,0.001028,0.000000,-0.005975,...,0.00000,-0.019899,0.000000,-0.014620,0.000000,0.015653,0.000000,0.028271,0.000000,0.026160


In [151]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming you have X and y pandas DataFrames

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize the logistic regression model
model = LogisticRegression()

# Step 3: Fit the model to the training data
model.fit(X_train, y_train)

# Step 4: Predict the labels for the test data
y_pred = model.predict(X_test)

# Step 5: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7222222222222222


In [150]:
preprocessed_without_fill.head()

,Unnamed: 0,umich_user_id,answer_count,average_answer_length,total_votes_given_answers,total_votes_received_answers,question_count,average_question_length,total_votes_given_questions,total_votes_received_questions,...,a758,a759,a760,a761,a762,a763,a764,a765,a766,a767
0,0,d8494a39cae7ac5417d491df43f8296a285b3cd6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,9c3e969ace3d1278a5e3ad42fa80dfc20379aaf5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,c91dab5c98bead9e683d15e9e2cc2c4584a8e626,10.0,797.200000,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.010755,0.018048,-0.067982,-0.010930,-0.008356,-0.007772,-0.006473,0.008379,0.006640,-0.005703
3,3,2246b3c08e2f3ee5d17f125ccc70baf3691667f9,5.0,589.000000,1.0,1.0,1.0,573.0,0.0,4.0,...,0.001105,0.008145,-0.050516,0.003159,-0.005324,-0.019285,-0.009427,0.008880,0.013980,0.018785
4,4,1f8331ad9f0125b0fc85a1c4ff28f9c198b6fca2,11.0,300.818182,3.0,3.0,0.0,0.0,0.0,3.0,...,0.000683,0.024056,-0.053009,0.000660,0.005344,-0.008130,-0.020281,0.003803,0.007443,0.014821
